In [31]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
import os
import tensorflow as tf
import numpy as np
import sqlite3

import findspark
findspark.init()

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#  Import and read the Ghibli_characters.csv
df = pd.read_csv("..\\project4\Resources\StudioGhibliCharacters - Sheet1.csv")
df.head()

,character name,age,height(cm),eyeColor,hairColor,specialPowers,country/placeOfResidence,gender,species,movie,releaseDate
0,Pazu,13.0,120.0,black,brown,0,Laputa,Male,Human,Castle in the Sky,1986
1,Lusheeta Toel Ul Laputa,13.0,120.0,black,black,1,Laputa,Female,Human,Castle in the Sky,1986
2,Dola,60.0,NaN,black,peach,0,Laputa,Female,Human,Castle in the Sky,1986
3,Romka Palo Ul Laputa,33.0,NaN,black,brown,0,Laputa,Male,Human,Castle in the Sky,1986
4,Uncle Pom,NaN,NaN,black,white,0,Laputa,Male,Human,Castle in the Sky,1986


In [44]:
Path('ghib_data.db').touch()

# Connect to SQLite database 
conn = sqlite3.connect('ghib_data.db') 
  
# Load CSV data into Pandas DataFrame 
ghib = pd.read_csv('..\\project4\Resources\StudioGhibliCharacters - Sheet1.csv') 
# Write the data to a sqlite table 
ghib.to_sql('sg_char', conn, if_exists='replace', index=False) 
  
# Create a cursor object 
cur = conn.cursor() 
# Fetch and display result 
for row in cur.execute('SELECT * FROM sg_char'): 
    print(row) 

('Pazu', 13.0, 120.0, 'black', 'brown', 0, 'Laputa', 'Male', 'Human', 'Castle in the Sky', 1986)
('Lusheeta Toel Ul Laputa', 13.0, 120.0, 'black', 'black', 1, 'Laputa', 'Female', 'Human', 'Castle in the Sky', 1986)
('Dola', 60.0, None, 'black', 'peach', 0, 'Laputa', 'Female', 'Human', 'Castle in the Sky', 1986)
('Romka Palo Ul Laputa', 33.0, None, 'black', 'brown', 0, 'Laputa', 'Male', 'Human', 'Castle in the Sky', 1986)
('Uncle Pom', None, None, 'black', 'white', 0, 'Laputa', 'Male', 'Human', 'Castle in the Sky', 1986)
('General Muoro', None, None, 'black', 'bald', 0, 'Laputa', 'Male', 'Human', 'Castle in the Sky', 1986)
('Duffi', None, None, 'dark brown', 'dark brown', 0, 'Laputa', 'Male', 'Human', 'Castle in the Sky', 1986)
('Louis', 30.0, None, 'dark brown', 'dark brown', 0, 'Laputa', 'Male', 'Human', 'Castle in the Sky', 1986)
('Charles ', None, None, 'dark brown', 'light brown', 0, 'Laputa', 'Male', 'Human', 'Castle in the Sky', 1986)
('Henri', None, None, 'dark brown', 'reddish 

In [62]:
# Query database for all distinct species and their counts
species_query = """
SELECT DISTINCT species, COUNT(species)
FROM sg_char
GROUP BY species;
"""

all_species = pd.read_sql_query(species_query, conn)
all_species

,species,COUNT(species)
0,Arch-Mage,1
1,Bakeneko,1
2,Boar God,1
3,Borrower,5
4,Cat,7
5,Deity,1
6,Dog,1
7,Dragon/Human,1
8,Fire Demon,1
9,Fish/Human,1


In [65]:
# Query database for Males vs Females
male_query = """
SELECT gender, COUNT(gender)
FROM sg_char
GROUP BY gender;
"""

male_characters = pd.read_sql_query(male_query,conn)
male_characters

,gender,COUNT(gender)
0,None,0
1,Female,84
2,Male,98


In [66]:
# Query database for all characters that share same eye color
eyeColor_query = """
SELECT eyeColor, COUNT(eyeColor)
FROM sg_char
GROUP BY eyeColor;
"""

same_eyeColors = pd.read_sql_query(eyeColor_query,conn)
same_eyeColors

,eyeColor,COUNT(eyeColor)
0,None,0
1,black,78
2,black,2
3,blue,5
4,brown,36
5,brown,1
6,dark brown,32
7,emerald,1
8,green,4
9,green grey,1


In [67]:
# Query database for all characters that share hair color
hairColor_query = """
SELECT hairColor, COUNT(hairColor)
FROM sg_char
GROUP BY hairColor;
"""

same_hairColors = pd.read_sql_query(hairColor_query,conn)
same_hairColors

,hairColor,COUNT(hairColor)
0,None,0
1,auburn,3
2,bald,6
3,beige,2
4,beige white,1
5,black,21
6,blonde,6
7,blue,1
8,brown,65
9,brown striped,1


In [71]:
# Query how many characters are in a movie
charInMovie = """
SELECT movie, COUNT('character name') as numberOfCharacters
FROM sg_char
GROUP BY movie;
"""

movies = pd.read_sql_query(charInMovie,conn)
movies

,movie,numberOfCharacters
0,Castle in the Sky,13
1,From Up on Poppy Hill,8
2,Grave of the Fireflies,6
3,Howl's Moving Castle,9
4,Kiki's Delivery Service,6
5,My Neighbor Totoro,10
6,My Neighbors the Yamadas,5
7,Nausicaa of the Valley of the Wind,13
8,Only Yesterday,10
9,Pom Poko,6


In [72]:
# Close connection to SQLite database 
conn.close() 

In [3]:
# clean dataframe and drop non-beneficial 
cleanedCharacters = df.drop(columns = ['character name','movie', 'releaseDate', 'country/placeOfResidence','eyeColor'], axis=1)
cleanedCharacters.head(20)

,age,height(cm),hairColor,specialPowers,gender,species
0,13.0,120.0,brown,0,Male,Human
1,13.0,120.0,black,1,Female,Human
2,60.0,NaN,peach,0,Female,Human
3,33.0,NaN,brown,0,Male,Human
4,NaN,NaN,white,0,Male,Human
5,NaN,NaN,bald,0,Male,Human
6,NaN,NaN,dark brown,0,Male,Human
7,30.0,NaN,dark brown,0,Male,Human
8,NaN,NaN,light brown,0,Male,Human
9,NaN,NaN,reddish brown,0,Male,Human


In [4]:
characters_df = cleanedCharacters.replace(np.nan, 0)
characters_df.head(20)

,age,height(cm),hairColor,specialPowers,gender,species
0,13.0,120.0,brown,0,Male,Human
1,13.0,120.0,black,1,Female,Human
2,60.0,0.0,peach,0,Female,Human
3,33.0,0.0,brown,0,Male,Human
4,0.0,0.0,white,0,Male,Human
5,0.0,0.0,bald,0,Male,Human
6,0.0,0.0,dark brown,0,Male,Human
7,30.0,0.0,dark brown,0,Male,Human
8,0.0,0.0,light brown,0,Male,Human
9,0.0,0.0,reddish brown,0,Male,Human


In [9]:
# Scale price data, return, and variance values
data_scaled = StandardScaler().fit_transform(
    characters_df[["age","height(cm)"]]
)

In [10]:
# Create a DataFrame with the scaled data
df_data_scaled = pd.DataFrame(
    data_scaled,
    columns=[["age","height(cm)"]]
)

# Display sample data
df_data_scaled.head()

,age,height(cm)
0,-0.191102,1.517909
1,-0.191102,1.517909
2,0.179696,-0.504138
3,-0.033315,-0.504138
4,-0.293663,-0.504138


In [13]:
# get_dummies to convert categorical values to numerical values
cat_dum = cleanedCharacters.copy()
cat_dum.drop(columns=["age","species","height(cm)"], axis=1, inplace=True)
cat_dum2 = pd.concat([cat_dum, df_data_scaled], axis=1)
ghibli_dummies = pd.get_dummies(cat_dum2)
ghibli_dummies.head()

,specialPowers,"(age,)","(height(cm),)",hairColor_auburn,hairColor_bald,hairColor_beige,hairColor_beige white,hairColor_black,hairColor_blonde,hairColor_blue,...,hairColor_purple,hairColor_red,hairColor_red,hairColor_reddish brown,hairColor_russian blue,hairColor_white,hairColor_whtie,hairColor_yellow,gender_Female,gender_Male
0,0,-0.191102,1.517909,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,-0.191102,1.517909,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
2,0,0.179696,-0.504138,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,0,-0.033315,-0.504138,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,0,-0.293663,-0.504138,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True


In [14]:
print(ghibli_dummies)

     specialPowers    (age,)  (height(cm),)  hairColor_auburn  hairColor_bald  \
0                0 -0.191102       1.517909             False           False   
1                1 -0.191102       1.517909             False           False   
2                0  0.179696      -0.504138             False           False   
3                0 -0.033315      -0.504138             False           False   
4                0 -0.293663      -0.504138             False           False   
..             ...       ...            ...               ...             ...   
179              0 -0.167434      -0.504138             False           False   
180              0 -0.167434      -0.504138              True           False   
181              0 -0.293663      -0.504138             False           False   
182              0 -0.096430      -0.504138             False           False   
183              0 -0.293663      -0.504138             False           False   

     hairColor_beige  hairC

In [15]:
# Look at Species value counts for binning
species_class = df['species']
species_class.head()

0    Human
1    Human
2    Human
3    Human
4    Human
Name: species, dtype: object

In [17]:
# Create All Human species classifications to be replaced with 1
Humans = species_class[species_class == "Human"]

# Replace in dataframe
for species in Humans:
    df['species'] = df['species'].replace(species,"1")

# Check to make sure binning was successful
df['species']

0              1
1              1
2              1
3              1
4              1
         ...    
179    New Human
180    New Human
181    New Human
182    New Human
183    New Human
Name: species, Length: 184, dtype: object

In [18]:
# Create All Human species classifications to be replaced with 1
OtherSpecies = species_class[species_class != "Human"]

# Replace in dataframe
for species in OtherSpecies:
    df['species'] = df['species'].replace(species,"0")

# Check to make sure binning was successful
df['species']

0      1
1      1
2      1
3      1
4      1
      ..
179    0
180    0
181    0
182    0
183    0
Name: species, Length: 184, dtype: object

In [19]:
# split data into features and target arrays
y = df['species'].astype('float64')
X = ghibli_dummies.values

In [20]:
# show first 3 X values
X[:3]

array([[0, -0.1911015378102931, 1.5179092379321943, False, False, False,
        False, False, False, False, True, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, True],
       [1, -0.1911015378102931, 1.5179092379321943, False, False, False,
        False, True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, True, False],
       [0, 0.1796963304830465, -0.5041381812860772, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, True, False, False, False, False, False, False,
        False, False, True, False]], dtype=object)

In [21]:
# show 10 y values
y[:20]

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    0.0
Name: species, dtype: float64

In [22]:
# split data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
print("Training Data Info")
print("Training Data Shape:", X_train.shape)
print("Training Data Labels Shape:", y_train.shape)

Training Data Info
Training Data Shape: (138, 37)
Training Data Labels Shape: (138,)


In [23]:
# Create Standard Scaler
scaler = StandardScaler()

# Fit scaler to training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu', input_dim=37))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 5)                 190       
                                                                 
 dense_5 (Dense)             (None, 10)                60        
                                                                 
 dense_6 (Dense)             (None, 20)                220       
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 491 (1.92 KB)
Trainable params: 491 (1.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
5/5 [==============================] - 2s 2ms/step - loss: 0.6847 - accuracy: 0.6014
Epoch 2/100
5/5 [==============================] - 0s 3ms/step - loss: 0.6711 - accuracy: 0.6739
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6587 - accuracy: 0.6739
Epoch 4/100
5/5 [==============================] - 0s 5ms/step - loss: 0.6475 - accuracy: 0.6812
Epoch 5/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6361 - accuracy: 0.7464
Epoch 6/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6246 - accuracy: 0.7754
Epoch 7/100
5/5 [==============================] - 0s 5ms/step - loss: 0.6147 - accuracy: 0.7826
Epoch 8/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6027 - accuracy: 0.7826
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 0.5911 - accuracy: 0.7754
Epoch 10/100
5/5 [==============================] - 0s 4ms/step - loss: 0.5788 - accuracy: 0.7971
Epoch 11/100
5/5 [===========

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.3840 - accuracy: 0.8478 - 265ms/epoch - 133ms/step
Loss: 0.38401877880096436, Accuracy: 0.8478260636329651
